# Monitor or cancel a job

View a list of your workloads on the [Workloads page](https://quantum.cloud.ibm.com/workloads).

## View job status

Go to your [Workloads table](https://quantum.cloud.ibm.com/workloads) and check under the Status column for whether a job has completed or failed.

## View remaining usage

Go to your [Instances table](https://quantum.cloud.ibm.com/instances) and select the tab associated with the plan you want to view remaining usage for. Total time used and total time remaining on your plan is displayed.

## View metrics on number of jobs and workloads submitted

Go to the [Analytics page](https://quantum.cloud.ibm.com/analytics) to see the total number of jobs submitted, as well as a count of batch workloads and session workloads. Note that you can only see the Analytics page for accounts that you own or manage.

## Monitor a job

Use the job instance to check the job status or retrieve the results by calling the appropriate command:

|                               |                                                                                                                                                                                                              |
| ----------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| job.result()                  | Review job results immediately after the job completes. Job results are available after the job completes. Therefore, job.result() is a blocking call until the job completes.                               |
| job.job\_id()                 | Return the ID that uniquely identifies that job. Retrieving the job results at a later time requires the job ID. Therefore, it is recommended that you save the IDs of jobs you might want to retrieve later. |
| job.status()                  | Check the job status.                                                                                                                                                                                        |
| job = service.job(\<job\_id>) | Retrieve a job you previously submitted. This call requires the job ID.                                                                                                                                      |

<span id="retrieve-later"></span>
## Retrieve job results at a later time

Call `service.job(\<job\_id>)` to retrieve a job you previously submitted. If you don't have the job ID, or if you want to retrieve multiple jobs at once; including jobs from retired QPUs (quantum processing units), call `service.jobs()` with optional filters instead. See [QiskitRuntimeService.jobs](../api/qiskit-ibm-runtime/qiskit-runtime-service#jobs).

<Admonition type="note" title="Deprecated provider packages">
  `service.jobs()` also returns jobs run from the deprecated `qiskit-ibm-provider` package. Jobs submitted by the older (also deprecated) `qiskit-ibmq-provider` package are no longer available.
</Admonition>

### Example

This example returns the 10 most recent runtime jobs that were run on `my_backend`:

In [ ]:
# This cell is hidden from users
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.transpiler import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
import numpy as np


my_backend = "ibm_torino"
service = QiskitRuntimeService()
# backend = service.backend(my_backend)
backend = service.least_busy()

# Define two circuits, each with one parameter with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.cx(0, 1)
circuit.h(0)
circuit.measure_all()


pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)

params = np.random.uniform(size=(2, 3)).T

sampler_pub = (transpiled_circuit, params)

# Instantiate the new estimator object, then run the transpiled circuit
# using the set of parameters and observables.
sampler = SamplerV2(mode=backend)
job = sampler.run([sampler_pub], shots=4)
print(job.job_id())

In [92]:
result = job.result()


spans = job.result().metadata["execution"]["execution_spans"]
print(spans)

ExecutionSpans([DoubleSliceSpan(<start='2025-05-13 21:09:41', stop='2025-05-13 21:09:42', size=24>)])


In [90]:
params = np.random.uniform(size=(2, 3))
params

array([[0.59989399, 0.01375162, 0.16726117],
       [0.01981301, 0.42679907, 0.88858636]])

In [93]:
mask = spans[0].mask(0)
mask

array([[[ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True]]])

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize the account first.
service = QiskitRuntimeService()
# Use `limit` to retrieve a specific number of jobs. The default `limit` is 10.
service.jobs(backend_name=my_backend)

## Cancel a job

You can cancel a job from the IBM Quantum Platform dashboard either on the Workloads page or the details page for a specific workload. On the Workloads page, click the overflow menu at the end of the row for that workload, and select Cancel. If you are on the details page for a specific workload, use the Actions dropdown at the top of the page, and select Cancel.

In Qiskit, use `job.cancel()` to cancel a job.

<span id="execution-spans"></span>
## View Sampler execution spans

The results of [`SamplerV2`](/docs/api/qiskit-ibm-runtime/sampler-v2) jobs executed in Qiskit Runtime contain execution timing information in their metadata.
This timing information can be used to place upper and lower timestamp bounds on when particular shots were executed on the QPU.
Shots are grouped into [`ExecutionSpan`](/docs/api/qiskit-ibm-runtime/execution-span-execution-span) objects, each of which indicates a start time, a stop time, and a specification of which shots were collected in the span.

An execution span specifies which data was executed during its window by providing an [`ExecutionSpan.mask`](/docs/api/qiskit-ibm-runtime/execution-span-execution-span#mask) method. This method, given any [Primitive Unified Block (PUB)](/docs/guides/primitive-input-output) index, returns a boolean mask that is `True` for all shots executed during its window. PUBs are indexed by the order in which they were given to the Sampler run call. If, for example, a PUB has shape `(2, 3)` and was run with four shots, then the mask's shape is `(2, 3, 4)`. See the [execution_span](/docs/api/qiskit-ibm-runtime/execution-span) API page for full details.

Example:

To view execution span information, review the metadata of the result returned by `SamplerV2`, which comes in the form of an `ExecutionSpans` object. This object is a list-like container containing subclass instances of `ExecutionSpan`s such as `SliceSpan`:

In [ ]:
from qiskit.primitives import BitArray

# Get the mask of the 1st PUB for the 0th span.
mask = spans[0].mask(0)

# Decide whether the 0th shot of parameter set (1, 2) occurred in this span.
in_this_span = mask[1, 2, 0]

# Create a new bit array containing only the PUB-1 data collected during this span.
bits = result[1].data.meas
filtered_data = BitArray(bits.array[mask], bits.num_bits)

Execution spans can be filtered to include information pertaining to specific PUBs, selected by their indices:

In [ ]:
# take the subset of spans that reference data in PUBs 0 or 2
spans.filter_by_pub([0, 2])

View global information about the collection of execution spans:

In [ ]:
print("Number of execution spans:", len(spans))
print("  Start of the first span:", spans.start)
print("     End of the last span:", spans.stop)
print("       Total duration (s):", spans.duration)

Extract and inspect a particular span:

In [ ]:
spans.sort()
print(" Start of first span:", spans[0].start)
print("   End of first span:", spans[0].stop)
print("#shots in first span:", spans[0].size)

<Admonition type="note">
It is possible for time windows specified by distinct execution spans to overlap. This is not because a QPU was performing multiple executions at once, but is instead an artifact of certain classical processing that might happen concurrently with quantum execution. The guarantee being made is that the referenced data definitely occurred in the reported execution span, but not necessarily that the limits of the time window are as tight as possible.
</Admonition>

## Next steps

<Admonition type="tip" title="Recommendations">
    - Try the [Grover's algorithm](/docs/tutorials/grovers-algorithm) tutorial.
    - [Migrate from qiskit_ibm_provider](/docs/migration-guides/qiskit-runtime-from-ibm-provider).
</Admonition>